In [45]:
%matplotlib inline
# imports required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pynetlogo
import numpy as np
import sys
sys.path.append('../')

from LandLab_experiments.ecohydr_mod import EcoHyd

In [34]:
def setUpNetLogoModel(leadFarmers, desperation, jealousy, grace):
    # think this is for the GUI idk?
    sns.set_style("white")
    sns.set_context("talk")

    # starts a NetLogo link, point the netlogo home path to where you have it installed on your machine
    netlogo = pynetlogo.NetLogoLink(
        gui=True,
        netlogo_home="/Volumes/NetLogo 6.3.0/NetLogo 6.3.0"
    )

    # loads a .nlogo model from provided path
    netlogo.load_model("/Users/mabelwylie/Documents/NetLogo-MOSS-Python/CouplingRepo/TreesAndThat/Coupling/modelv1.nlogo")

     # sets globals
    globals = "update-globals " + str(leadFarmers) + " " + str(desperation) + " " + str(jealousy) + " " + str(grace)
    netlogo.command(globals)
    
    # runs the model setup command
    netlogo.command("setup")

    return netlogo

In [35]:
def reportsToDataFrame(netlogo):
    # gets field attributes and puts it in a data frame
    fieldAttributes = netlogo.report("get-info")
    sorted_list = sorted(fieldAttributes, key=lambda x: (-x[1],x[0]))
    fieldData = pd.DataFrame(columns=["who", "xcor","ycor","owner-id","implements-WSA", "yield"], data=sorted_list)
    return fieldData

In [36]:
def convertWSAToNPArray(data):
    # sets bool into correct format to pass to hydrology model
    emptynp = np.empty((101,101))
    for i in range(0,101):
        for j in range(0,101):
            ycorFromIndex = i * -1 + 50
            xcorFromIndex = j - 50
            emptynp[i][j]= data.loc[(data['ycor'] == ycorFromIndex) & (data['xcor'] == xcorFromIndex)]["implements-WSA"].iloc[0]
    return emptynp

In [37]:
def convertHydrologyToDF(hydrologyArray, data):
    # method converts the hydrology model output into a pandas dataframer
    flattenedArray = hydrologyArray.flatten()
    # 2 is the placeholder to multiply the biomass to get yield
    data["yield"] = flattenedArray * 2
    return data

In [46]:
def fullModelRun(leadFarmers, desperation, jealousy, grace, fileSuffix):
    # sets up model
    netlogo = setUpNetLogoModel(leadFarmers, desperation, jealousy, grace)

    # gets base field attributes and puts it in a data frame
    fieldData = reportsToDataFrame(netlogo)

    # make dataframe to record all changes
    bestData = fieldData.copy()
    
    for year in range(0,10):
        # run for 10 years

        # runs one step of social model
        netlogo.command("farming-year")

        # converts field data to a df
        fieldData = reportsToDataFrame(netlogo)

        # converts the usingWSA bool for each field into an NP array
        hydrologyInput = convertWSAToNPArray(fieldData)

        # use dummy model
        hydrologyOutput = hydrologyModelYear(hydrologyInput)

        # trying to implement the hydrology model
        # input to this block will be the WSA np.array, with 0 or 1 corresponding to if the field is using WSA
        # the output of this block should be a np.array with biomass
        
        #Ecohyd_model = EcoHyd(WSA_array=hydrologyInput)
        #EcoHyd.stepper(Ecohyd_model)

        # converts the updated yields to the dataframe
        convertHydrologyToDF(hydrologyOutput,fieldData)

        # writes this new yield information to the netlogo implementation
        netlogo.write_NetLogo_attriblist(fieldData, "field")

        

## dummy hydrology model to test running
def hydrologyModelYear(input):
    print("One year yield")
    return np.zeros((101,101))

fullModelRun(20, 0.5, 0.5, 3, 1)

One year yield


FieldError: Unable to add the field, 'rainfall__flux', to the group, 'grid', because a field with that name already exists in that group. Use `clobber=True` to replace the existing field. For example, grid.add_field('rainfall__flux', at='grid', clobber=True)